# One cluster per annotation

In [27]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import math

In [2]:
an_df = pd.read_csv('annotated_25k_cluster_df.csv').drop('Unnamed: 0', axis=1)

In [3]:
an_df

,count_bindings_in_miRNA,pair_state_No,mFE,hairpin_count,stem_length,distanceToloop,percentage_PairedInMiRNA,pair_state_Yes,binding_count,armType_loop,interiorLoopCount,index,clusterName,chrom,start,end,miRNA,sample,annotation
0,20,0,-42.9,1,34,5,0.95,1,34,0,6,0,mapped_mirna_ERR038410:miRCluster_1_22:chr1:11...,1,1167124,1167145,True,ERR038410,Hsa-Mir-8-P1a_pri
1,18,0,-40.9,1,32,6,0.90,1,32,0,5,1,mapped_mirna_ERR038410:miRCluster_2_24:chr1:11...,1,1167160,1167183,True,ERR038410,Hsa-Mir-8-P1a_pri
2,18,1,-45.5,1,32,12,0.78,0,32,0,4,2,mapped_mirna_ERR038410:miRCluster_3_23:chr1:11...,1,1167916,1167938,True,ERR038410,Hsa-Mir-8-P2a_pri
3,17,1,-38.8,1,29,3,0.77,0,29,0,3,3,mapped_mirna_ERR038410:miRCluster_4_22:chr1:11...,1,1169055,1169076,True,ERR038410,Hsa-Mir-8-P3a_pri
4,18,1,-39.2,1,35,9,0.90,0,35,0,5,4,mapped_mirna_ERR038410:miRCluster_31_23:chr1:9...,1,9151734,9151756,True,ERR038410,Hsa-Mir-34-P1_pri
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23802,11,1,-14.1,1,11,-17,0.55,0,11,1,2,24091,mapped_nonMirna_SRR944034:miRCluster_9783_22:c...,X,9912443,9912464,False,SRR944034,NaN
23803,14,1,-19.9,2,11,-9,0.82,0,11,1,0,24092,mapped_nonMirna_SRR944034:miRCluster_10009_19:...,X,43656244,43656262,False,SRR944034,NaN
23804,11,1,-35.8,2,33,-23,0.73,0,33,0,6,24093,mapped_nonMirna_SRR944034:miRCluster_10079_17:...,X,130087016,130087032,False,SRR944034,NaN
23805,10,1,-11.5,1,9,-18,0.53,0,9,1,1,24094,mapped_nonMirna_SRR944034:miRCluster_9927_23:c...,X,135032499,135032521,False,SRR944034,NaN


... I think we should redo this annotation process, just to make sure. 

In [4]:
data = pd.read_csv('SVM_training/all_tissues_mapped_dataset_15_refined_miRGeneDB_total_features_updated.csv')

In [5]:
featureListSelected = []
with open('SVM_training/selected_features.txt', 'r') as inf:
    line = inf.readline()
    line = inf.readline()
    while line != '':
        featureListSelected.append((float(line.strip().split('\t')[1]), int(
            line.strip().split('\t')[2]), line.strip().split('\t')[0]))
        line = inf.readline()

In [6]:
data_no_cn = data.drop(data.columns[[1, 2]], axis=1)
data_dummies = pd.get_dummies(data_no_cn)
data_dummies = data_dummies.reset_index()[[col for col in data_dummies] + ['index']]
#featureList = data.columns.values.tolist()[1:]                                                                                                                                                                                                                       

# subselect data with the selected features                                                                                                                                                                                                                           
totalfeatureList = data_dummies.columns.values.tolist()
subIndexList = [totalfeatureList.index(
    item[2]) for item in featureListSelected] + [len(data_dummies.columns) - 1]
# Creat a variable for the feature data                                                                                                                                                                                                                               
data_x = data_dummies.iloc[:, subIndexList].values
# Create a variable for the target data                                                                                                                                                                                                                               
data_y = data_dummies.iloc[:, 0].values

df_cn = data_dummies.iloc[:, subIndexList].merge(data['clusterName'], left_index=True, right_index=True)

In [7]:
df_cn

,count_bindings_in_miRNA,exactMatchRatio,pair_state_No,mFE,head_minus3_TemplateNucleotide_percentage,hairpin_count,stem_length,distanceToloop,percentage_PairedInMiRNA,headUnstableLength,...,binding_count,tail_plus1_A_percentage,armType_loop,tail_plus3_A_percentage,tail_plus5_TemplateNucleotide_percentage,tail_plus1_TemplateNucleotide_percentage,interiorLoopCount,head_minus1_TemplateNucleotide_percentage,index,clusterName
0,20,0.636364,0,-42.9,0.0,1,34,5,0.95,0,...,34,0.000000,0,0.0,0.0,1.000000,6,0.0,0,mapped_mirna_ERR038410:miRCluster_1_22:chr1:11...
1,18,0.327044,0,-40.9,0.0,1,32,6,0.90,2,...,32,0.625000,0,0.0,0.0,0.936000,5,1.0,1,mapped_mirna_ERR038410:miRCluster_2_24:chr1:11...
2,18,0.565957,1,-45.5,0.0,1,32,12,0.78,0,...,32,0.170455,0,0.0,0.0,0.000000,4,0.0,2,mapped_mirna_ERR038410:miRCluster_3_23:chr1:11...
3,17,0.925926,1,-38.8,0.0,1,29,3,0.77,0,...,29,0.000000,0,0.0,0.0,0.000000,3,0.0,3,mapped_mirna_ERR038410:miRCluster_4_22:chr1:11...
4,18,0.828571,1,-39.2,0.0,1,35,9,0.90,0,...,35,0.000000,0,1.0,0.0,1.000000,5,0.0,4,mapped_mirna_ERR038410:miRCluster_31_23:chr1:9...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24091,11,1.000000,1,-14.1,0.0,1,11,-17,0.55,0,...,11,0.000000,1,0.0,0.0,1.000000,2,0.0,24091,mapped_nonMirna_SRR944034:miRCluster_9783_22:c...
24092,14,0.100000,1,-19.9,0.0,2,11,-9,0.82,1,...,11,0.000000,1,0.0,0.0,0.777778,0,1.0,24092,mapped_nonMirna_SRR944034:miRCluster_10009_19:...
24093,11,0.086957,1,-35.8,0.0,2,33,-23,0.73,2,...,33,0.000000,0,0.0,0.0,1.000000,6,0.4,24093,mapped_nonMirna_SRR944034:miRCluster_10079_17:...
24094,10,1.000000,1,-11.5,1.0,1,9,-18,0.53,4,...,9,0.000000,1,0.0,0.0,0.000000,1,1.0,24094,mapped_nonMirna_SRR944034:miRCluster_9927_23:c...


In [8]:
def parse_cluster_name(row):
    row['chrom'] = re.search('(?<=chr)[^:]*', row['clusterName']).group(0)
    row['start'] = int(re.search('(?<=chr).*(?=_)', row['clusterName']).group(0).split(':')[1])
    row['end'] = int(re.sub('(\+|\-)', '', re.search('(?<=_)[^_]+$', row['clusterName']).group(0)))
    if row['chrom'] in [str(int_ + 1) for int_ in range(22)] + ['X', 'Y']:
        row['good_chrom'] = True
    else:
        row['good_chrom'] = False
    if re.search('(?<=mapped_)[^_]*', row['clusterName']).group(0) == 'mirna':
        row['miRNA'] = True
    else:
        row['miRNA'] = False
    row['sample'] = re.search('(S|E)RR[^\:]*', row['clusterName']).group(0)
    return row

df_cn = df_cn.apply(parse_cluster_name, axis=1)

In [9]:
df = df_cn[df_cn['good_chrom']].drop('good_chrom', axis=1)

In [10]:
hsa_bed = pd.read_csv('mirgenedb/hsa-all.bed', sep='\t', header=None, names=['chrom', 'start', 'end', 'annotation', 'orientation', 'strand'])

In [11]:
def add_annotation(row, mirgene_df):
    df_chrom = mirgene_df[[chrom[3:] == row['chrom'] for chrom in mirgene_df['chrom']]]
    for start, end, annotation in zip(df_chrom['start'], df_chrom['end'], df_chrom['annotation']):
        if abs(row['start'] - start) < 5 and abs(row['end']- end) < 5:
            row['annotation'] = annotation
            return row
    row['annotation'] = float('NaN')
    return row

In [12]:
df = df.apply(add_annotation, mirgene_df = hsa_bed, axis=1)

In [16]:
df.to_csv('annotated_cluster_df_31_May.tsv', sep='\t', index=None)

In [17]:
[ann for ann in df['annotation'].dropna().tolist() if '3p' not in ann and '5p' not in ann]

[]

In [19]:
def strip_3p_5p(ann):
    precursor = re.sub('(_3p\*?|_5p\*?)', '', ann)
    drop_version = re.sub('-v[0-9]', '', precursor)
    return drop_version

In [20]:
miRNAs = df[~df['annotation'].isna()].copy()

In [21]:
miRNAs['precursor'] = [strip_3p_5p(ann) for ann in miRNAs['annotation']]

In [22]:
unique_precursors = []
for ann in set(miRNAs['precursor']):
    unique_precursors.append(miRNAs[miRNAs['precursor'] == ann].sample(n=1, random_state=1))
unique_precursor_df = pd.concat(unique_precursors)

In [23]:
len(set(unique_precursor_df['precursor']))

439

In [24]:
len(unique_precursor_df)

439

In [38]:
%%bash
pwd

/home/jfernand/mirge/miRge3.0-feature-extraction


In [25]:
non_mirna = df[~df['miRNA']]

In [28]:
def annotate_non_mirna(row):
    if row['start'] % 1000 < 200:
        row['ann_start'] = math.floor(row['start'] / 1000) * 1000 - 1000
    else:
        row['ann_start'] = math.floor(row['start'] / 1000) * 1000
    if row['end'] % 1000 > 800:
        row['ann_end'] = math.ceil(row['end'] / 1000) * 1000 + 1000
    else:
        row['ann_end'] = math.ceil(row['end'] / 1000) * 1000
    return row

nmdf = non_mirna.apply(annotate_non_mirna, axis=1)

In [33]:
unique_non_mirnas = nmdf.loc[nmdf[['ann_start', 'ann_end']].drop_duplicates().index]

In [35]:
nmdf_439 = unique_non_mirnas.sample(n=len(unique_precursor_df), random_state=1)

In [40]:
nmdf_439.to_csv('one_cluster_per_annotation/negative_examples.csv')
unique_precursor_df.to_csv('one_cluster_per_annotation/positive_examples.csv')

In [121]:
nmdfs = []
for chrom in set(unique_precursor_df['chrom']):
    mirna_count = len(unique_precursor_df[unique_precursor_df['chrom'] == chrom])
        

0 19
1 22
2 9
3 2
4 10
5 11
6 X
7 16
8 14
9 18
10 6
11 5
12 1
13 12
14 4
15 13
16 7
17 21
18 3
19 8
20 15
21 17
22 20


In [120]:
seen_non_miRNAs = np.zeros((len(unique_precursor_df), 3))


In [58]:
seen_non_miRNAs.shape[0]

439

In [62]:
unique_non_miRNAs = non_mirna.sample(n=len(unique_precursor_df), random_state=1)

In [76]:
unique_non_miRNAs

,count_bindings_in_miRNA,exactMatchRatio,pair_state_No,mFE,head_minus3_TemplateNucleotide_percentage,hairpin_count,stem_length,distanceToloop,percentage_PairedInMiRNA,headUnstableLength,...,interiorLoopCount,head_minus1_TemplateNucleotide_percentage,index,clusterName,chrom,start,end,miRNA,sample,annotation
4370,13,0.001036,1,-16.2,0.000000,2,32,10,0.87,1,...,3,1.000000,4370,mapped_nonMirna_ERR973611:miRCluster_4684_16:c...,3,47649331,47649346,False,ERR973611,NaN
16082,12,0.057971,1,-35.3,0.000000,1,21,1,0.80,0,...,6,0.000000,16082,mapped_nonMirna_SRR372650:miRCluster_3989_16:c...,19,1108754,1108769,False,SRR372650,NaN
22194,2,0.277778,1,-18.5,0.785714,1,14,-8,0.20,9,...,2,0.722222,22194,mapped_nonMirna_SRR837842:miRCluster_8129_22:c...,12,48957429,48957450,False,SRR837842,NaN
13456,9,0.000627,1,-11.3,0.000000,1,14,-9,0.53,0,...,1,0.000000,13456,mapped_nonMirna_SRR333674:miRCluster_1962_17:c...,1,143596509,143596525,False,SRR333674,NaN
2616,13,0.102564,1,-20.1,0.000000,1,16,-12,0.81,2,...,3,1.000000,2616,mapped_nonMirna_ERR852097:miRCluster_2507_17:c...,12,57552048,57552064,False,ERR852097,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10821,9,0.006766,1,-27.4,0.000000,1,23,7,0.64,3,...,5,0.026477,10821,mapped_nonMirna_SRR2174513:miRCluster_11615_16...,19,1460790,1460805,False,SRR2174513,NaN
7422,13,0.190476,1,-24.6,0.666667,1,21,1,0.87,10,...,5,0.210526,7422,mapped_nonMirna_SRR1635903:miRCluster_48521_26...,19,34221624,34221649,False,SRR1635903,NaN
7859,11,0.785714,1,-28.2,0.666667,3,30,-64,0.79,7,...,4,1.000000,7859,mapped_nonMirna_SRR1635903:miRCluster_16489_28...,5,177523988,177524015,False,SRR1635903,NaN
7364,13,0.072727,1,-33.0,0.000000,2,27,0,0.68,0,...,2,0.000000,7364,mapped_nonMirna_SRR1635903:miRCluster_47505_24...,19,1453008,1453031,False,SRR1635903,NaN


In [89]:
chrom_dfs = []
for chrom in set(unique_non_miRNAs['chrom']):
    mini = unique_non_miRNAs[unique_non_miRNAs['chrom'] == chrom]
    drops = []
    for start, end in zip(mini['start'], mini['end']):
        start_distances = np.abs(np.concatenate((mini['start'].values, mini['end'].values)) - start)
        end_distances = np.abs(np.concatenate((mini['end'].values, mini['end'].values)) - end)
        if np.sort(start_distances)[1] < 1000 or np.sort(end_distances)[1] < 1000:
            drops.append(True)
        else:
            drops.append(False)
    mini['drop'] = drops
    chrom_dfs.append(mini)

/home/jfernand/anaconda3/envs/mirge-dev/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [91]:
unique_non_miRNAs = pd.concat(chrom_dfs)

In [96]:
testing = np.array([1, 3, 5])

In [99]:
np.setdiff1d(testing, [3, 5, 2])

array([1])

In [93]:
unique_non_miRNAs.groupby('drop').count()

,count_bindings_in_miRNA,exactMatchRatio,pair_state_No,mFE,head_minus3_TemplateNucleotide_percentage,hairpin_count,stem_length,distanceToloop,percentage_PairedInMiRNA,headUnstableLength,...,interiorLoopCount,head_minus1_TemplateNucleotide_percentage,index,clusterName,chrom,start,end,miRNA,sample,annotation
drop,,,,,,,,,,,,,,,,,,,,,
True,439,439,439,439,439,439,439,439,439,439,...,439,439,439,439,439,439,439,439,439,0


We need some way of clustering nonMirnas to drop duplicates in those examples. Maybe just by location? This is fine, just make sure you don't see the same start position twice (by a fudge factor of say 100) to generate as many non-miRNA examples as we have miRNA examples. 

In [85]:
print(set([type(value) for value in unique_non_miRNAs['end'].values]))

{<class 'numpy.int64'>}


In [88]:
np.concatenate((unique_non_miRNAs['start'].values, unique_non_miRNAs['end'].values), axis=0)

array([ 47649331,   1108754,  48957429, 143596509,  57552048, 149861137,
        32412214, 152300057, 210170429, 160291216,  34221631,  50426262,
        30484923,  79816103,  74505913,  54508702,  42659141, 115807202,
        94427215,  53348492, 145996261,  72198921,  93584887,  40348743,
        81442069,  72295730,  26202274,  61477979,  66295064,  70538077,
       147965091,  51751562,  44133626,  68755208,  79121574, 124415402,
        29288760,  92837289,  45784838,  25097665,  69655655, 219493145,
        28049562,  31754062,  23425746,  54772908, 183552152, 110256417,
       121800661, 102905873, 160139711,  31228275, 121297142,  34172485,
       182917341,  42061357, 108149301, 102040335, 120723342,   1511190,
         4760072,  17962712,  45308485,   5193675,  57684343,  44047103,
         4700759,  62852953,  20455263,  41232092,  21283957,   5622736,
       107005614,   3786404,  11022440,  73419587, 121440379, 124598828,
        60091996,  44158017,  72887433, 170819856, 

In [86]:
help(np.concatenate)

Help on function concatenate in module numpy:

concatenate(...)
    concatenate((a1, a2, ...), axis=0, out=None)
    
    Join a sequence of arrays along an existing axis.
    
    Parameters
    ----------
    a1, a2, ... : sequence of array_like
        The arrays must have the same shape, except in the dimension
        corresponding to `axis` (the first, by default).
    axis : int, optional
        The axis along which the arrays will be joined.  If axis is None,
        arrays are flattened before use.  Default is 0.
    out : ndarray, optional
        If provided, the destination to place the result. The shape must be
        correct, matching that of what concatenate would have returned if no
        out argument were specified.
    
    Returns
    -------
    res : ndarray
        The concatenated array.
    
    See Also
    --------
    ma.concatenate : Concatenate function that preserves input masks.
    array_split : Split an array into multiple sub-arrays of equal or
   